In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import pandas as pd
import geopandas as gpd

In [9]:
df = pd.read_csv('../data/final/gbd_cardiovascular_allAges_final.csv')

In [11]:
death_df = df[(df.measure_name == "Deaths")]

In [13]:
death_df = death_df.rename({'Country Code': 'iso_a3'}, axis=1)

In [18]:
death_df.head()

,measure_name,location_name,metric_name,year,val,iso_a3
0,Deaths,Global,Percent,1990,0.258754,GLB
1,Deaths,Global,Rate,1990,225.584376,GLB
2,Deaths,Timor-Leste,Percent,1990,0.080068,TLS
3,Deaths,Timor-Leste,Rate,1990,79.809246,TLS
4,Deaths,Japan,Percent,1990,0.348712,JPN


In [28]:
import sys
sys.path.append('..')
from src.plotting import custom_cmap

from tueplots import bundles
from tueplots.constants.color import rgb
from matplotlib.colors import LinearSegmentedColormap

In [59]:
blue = rgb.tue_blue
red = rgb.tue_red
# make red a bit lighter, but not towards pink, less blue
red = (red[0]*1.5, red[1] * 0.8, red[2] * 0.8)
# Create a colormap
cmap = LinearSegmentedColormap.from_list('blue_to_red', [blue, red])


In [76]:
# Load the world map data using geopandas
plt.rcParams.update(bundles.icml2022(column="full", ncols=2, nrows=1))

def plot_world_map(filename, data, country_col, measured_col, label, year):
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Merge your data with the geopandas DataFrame using country names
    merged = world.merge(data[data.year==year], how='left', left_on='iso_a3', right_on=country_col)

    # Fill NaN values (countries with no deaths) with 0
    merged[measured_col] = merged[measured_col].fillna(0)
    
    # Plotting the map
    # fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    ax = merged.plot(column=measured_col, cmap=cmap, linewidth=0.2, edgecolor='0.2', legend=True,
                legend_kwds={'label': label, 'orientation': "horizontal"}, vmin=0, vmax=1000,
               )
    ax.set_title(f'{label} of Ischemic heart disease by countries in year {year}')

    ax.axis('off')

    chart = ax.get_figure()
    chart.savefig(filename, dpi=600)
    plt.close()
    # return ax

In [78]:
filenames = []
for year in range(1990, 2020):
    filename = f'year_{year}.png'
    plot_world_map(filename, death_df, country_col='iso_a3', measured_col='val',  label="Death Rate", year=year)
    filenames.append(filename)

# Create gif
with imageio.get_writer('world_map_CVDs_by_year.gif', mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in filenames:
    os.remove(filename)